In [1]:
import glob
from pathlib import Path
from llama_index.readers.file import PDFReader
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from tqdm import tqdm

pdf_folder = "../data/raw/documents"
pdf_files = glob.glob(f"{pdf_folder}/*.pdf")
print(f"Founded {len(pdf_files)} file PDF.")



/home/dzakirm/anaconda3/envs/satdat2024/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-02-24 20:24:05.976446: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-24 20:24:06.033237: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-24 20:24:06.942301: W tensorflow/compiler/tf2tensorrt/utils/p

Founded 6 file PDF.


In [2]:
loader = PDFReader()

documents = []
for pdf_file in tqdm(pdf_files):
    file_path = Path(pdf_file)
    docs = loader.load_data(file=file_path)
    documents.extend(docs)
print(f"Total Extracted Documents: {len(documents)}")

model = SentenceTransformer('all-MiniLM-L6-v2')

sample_embedding = model.encode(documents[0].text)
vector_dim = len(sample_embedding)

client = QdrantClient(host="localhost", port=6333)
collection_name = "pdf_collection"
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_dim, distance=Distance.COSINE)
)


 67%|██████▋   | 4/6 [00:02<00:01,  1.92it/s]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 59 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing object 75 0 (offset 0)
Ignoring wrong pointing object 79 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong pointing object 97 0 (offset 0)
Ignoring wrong pointing object 109 0 (offset 0)
Ignoring wrong pointing object 111 0 (offset 0)
Ignoring wrong pointing object 113 0 (offset 0)
Ignoring wrong pointing object 128 0 (offset 0)
Ignoring wrong pointing object 171 0 (offset 0)
Ignoring wro

Total Extracted Documents: 91


/home/dzakirm/anaconda3/envs/satdat2024/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


True

In [3]:
points = []
doc_id = 0
for doc in tqdm(documents):
    text = doc.text
    embedding = model.encode(text)
    filename = getattr(doc, "extra_info", {}).get("file_name", "unknown")

    point = {
        "id": doc_id,
        "vector": embedding.tolist(),
        "payload": {
            "filename": filename,
            "text": text
        }
    }
    points.append(point)
    doc_id += 1

batch_size = 100
for i in tqdm(range(0, len(points), batch_size)):
    batch = points[i:i+batch_size]
    client.upsert(collection_name=collection_name, points=batch)

  0%|          | 0/91 [00:00<?, ?it/s]/tmp/ipykernel_53134/346957233.py:6: DeprecationWarning: Call to deprecated function (or staticmethod) extra_info. ('extra_info' is deprecated, use 'metadata' instead.) -- Deprecated since version 0.12.2.
  filename = getattr(doc, "extra_info", {}).get("file_name", "unknown")
100%|██████████| 1/1 [00:00<00:00, 24.71it/s]
